<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Naas Chat Plugin

## Input

### Import libraries

In [ ]:
import os
import glob
import pydash
import json
import naas
import naas_data_product

## Model

### Get connectors files

In [ ]:
def get_connectors(input_dir):
    # Init
    connectors = []
    dir_name = input_dir.split('/')[-1]
    directory = os.path.join(naas_data_product.INPUTS_PATH, "connections")
    files = sorted(glob.glob(os.path.join(directory, "**", "*.ipynb"), recursive=True))
    
    # Get all files
    if dir_name == "models":
        return files
    
    # Get connectors
    for file in files:
        # Open file and get cells
        with open(file) as f:
            nb = json.load(f)
        cells = nb.get("cells")
        tags = cells[2].get("source")[0].replace("**Tags:**", "").strip()
        tags = [f"{tag.strip()}" for tag in tags.split("#") if tag != ""]
        if dir_name in tags:
            connectors.append(file)
    return connectors

# input_dir = os.path.join(naas_data_product.MODELS_PATH)
# input_dir = os.path.join(naas_data_product.ROOT_PATH, "models", "content-engine")
# print(input_dir)
# connectors = get_connectors(input_dir)
# connectors

### Create Chat commands from directory

In [ ]:
def create_command(input_dir):
    # Init
    commands = []
    connectors = get_connectors(input_dir)
    files = sorted(glob.glob(os.path.join(input_dir, "**", "*.ipynb"), recursive=True) + connectors)

    # Loop on .ipynb files
    for file in files:
        # Init
        command = {}
        section = file.split("/")[-2]
        file_name = file.split("/")[-1]

        # Exclude file name chat plugin or pipeline
        if not file_name.startswith("__") and not file_name.endswith("__") and not "utils" in file:
#             # Create section
#             section_name = section.replace("-", " ").upper()
#             command_names = [command.get("name") for command in commands]
#             if section_name not in command_names:
#                 commands.append(
#                     {"name": section_name, "description": ""}
#                 )
                
            # Open file and get cells
            with open(file) as f:
                nb = json.load(f)
            cells = nb.get("cells")

            # Find tags #command to generate command
            is_command = False
            for cell in cells:
                source = cell.get("source")
                if "#command" in " ".join(source):
                    is_command = True
                    break

            if is_command:
                # Get name & webhook
                payload = {}
                command["name"] = file_name.replace(".ipynb", "")
                command["avatar"] = cells[0].get("source")[0].split('src="')[1].replace('"', '').replace('>', '')
                command["action"] =  {
                    "request_type": "POST",
                    "url": naas.webhook.add(file, params={"inline": True}),
                    "payload": {}
                }
                payload = {}

                # Add command description & payload
                for i, cell in enumerate(cells):
                    source = "".join(cell.get("source"))
                    tags = pydash.get(cell, "metadata.tags")
                    # Create description
                    if "Description" in source:
                        command_description = source.split("**Description:**")[1].strip()
                        command["description"] = command_description.replace('notebook', 'command')
                    # Create command payload
                    if tags and "variables" in tags or "Setup variables" in source:
                        variables = source
                        defaults = "".join(cells[i+1].get("source"))
                    if tags and "parameters" in tags:
                        for s in cells[i+1].get("source"):
                            if "=" in s:
                                parameter = s.split("=")[0].strip()
                                payload[parameter] = {
                                    "type": "str",
                                    "description": variables.split(f"`{parameter}`:")[-1].split("-")[0].replace("\n", "").strip(),
                                    "default": defaults.split(f"{parameter} =")[-1].split("\n")[0].strip(),
                                }
                command["action"]["payload"] = payload

                # Generate commands (to be develop)
                commands.append(command)
    print(json.dumps(commands, indent=4))
    return commands

# commands = create_command("/home/ftp/abi/models")
# commands

## Output